In this notebook, I will be tackling a project of creating a nairobi house price prediction model that will help determine the prices of houses in narobi with respect to the various features. This concept that we shall touch on today will be applicable to any house price prediction not just nairobi. I plan to use a very dirty dataset so tha we can get the chance to sharpen our logic senses, apply then concepts of mathematical calculus and statistics and also know how to tackle such a data in any other occurrence. I hope you enjoy. Lets do some coding

In [1]:
# below are the libraries we shall be using with their uses described.
"""
pandas for data manipulation and analysis.
numpy for numerical operations.
matplotlib.pyplot and seaborn for data visualization.
train_test_split from sklearn.model_selection to split the data into training and test sets.
LinearRegression from sklearn.linear_model for linear regression (you can use this for comparison with XGBoost).
mean_squared_error and r2_score from sklearn.metrics to evaluate the model performance.
xgboost for the XGBoost regression model.
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics


In [2]:
df = pd.read_csv("houses-for-sale.csv")
df

,title,location,size,selling price
0,6 Bed Villa with En Suite at Kiambu Road,Kiambu Road,"6bdrm Mansion in Mushroom Estate, Kiambu / Kia...","KSh 130,000,000"
1,4 Bed Townhouse with En Suite at Off Kiambu Road,"Off Kiambu Road, Kiambu Road",4 Bedroom with DSQ for Sale,"KSh 26,950,000"
2,4 Bed Villa with En Suite at Masai Lodge Road,"masai lodge road, Ongata Rongai",New modern villa for sale.,"KSh 16,000,000"
3,5 Bed House with En Suite in Ngong Road,Ngong Road,5 Bedroom Town House For Sale In Kilimani,"KSh 45,000,000"
4,5 Bed Townhouse with En Suite in Lavington,Lavington,Modern 5 Bedroom Townhouse For Sale,"KSh 72,000,000"
...,...,...,...,...
1871,5 Bed House with En Suite at Nandi Road,"Nandi road, Karen",5 BEDROOM HOUSE FOR SALE IN KAREN NAIROBI KENYA,"KSh 199,000,000"
1872,4 Bed House with En Suite in Kitengela,Kitengela,4 Bedroom Maisonette for Sale in Kitengela EPZ...,"KSh 16,000,000"
1873,4 Bed House with En Suite in Syokimau,Syokimau,4 BEDROOMS MAISONETTE IN SYOKIMAU.,"KSh 13,800,000"
1874,4 Bed Townhouse with Garden in Lavington,Lavington,4 bedroom townhouse for sale in Lavington,"KSh 79,000,000"


# Data overview
   - Obcervations
   - Sanity checks

In [3]:
# we first confirm for null values and were off for a good start.
df.isnull().sum()

title            0
location         0
size             0
selling price    0
dtype: int64

Just because there is no null values, this is not proof enough that our dataset is clean and we can start
working with it. Use more prompts to check for unique values, duplicates, dtypes, and at times desriptive
statustics. This tend to be very insightful from a mathematical and logical point of view.

In [4]:
df.dtypes

title            object
location         object
size             object
selling price    object
dtype: object

Notice that all, the features contain dtype object. We will change this later on to maje sure we have everything as the appropriate dtype.

In [5]:
df['title'].unique()

array(['6 Bed Villa with En Suite at Kiambu Road',
       '4 Bed Townhouse with En Suite at Off Kiambu Road',
       '4 Bed Villa with En Suite at Masai Lodge Road', ...,
       '3 Bed House with En Suite at Kona Ya Musa',
       '3 Bed House with En Suite in Kileleshwa',
       '5 Bed House with En Suite at Nandi Road'], dtype=object)

 You can notice how wranglled the tittle column is. we can can come up with almost more that two new features that can be very important in our analysis. In this case, I shall apply a regex function that willl beak the title column to bera new columns namels; number_of_bedrooms, house_type, location, and amenities. This will become very crucial during model training since they all determine the price of a house hold

In [6]:
df['selling price'].unique()


array(['KSh 130,000,000', 'KSh 26,950,000', 'KSh 16,000,000',
       'KSh 45,000,000', 'KSh 72,000,000', 'KSh 46,000,000',
       'KSh 85,000,000', 'KSh 50,000,000', 'KSh 75,000,000',
       'KSh 500,000,000', 'KSh 140,000,000', 'KSh 15,000,000',
       'KSh 60,000,000', 'KSh 55,000,000', 'Price not communicated',
       'KSh 427,000,000', 'KSh 65,000,000', 'KSh 48,000,000',
       'KSh 88,000,000', 'KSh 27,000,000', 'KSh 150,000,000',
       'KSh 120,000,000', 'KSh 110,000,000', 'KSh 90,000,000',
       'KSh 35,000,000', 'KSh 59,000,000', 'KSh 95,000,000',
       'KSh 24,000,000', 'KSh 20,000,000', 'KSh 23,000,000',
       'KSh 115,000,000', 'KSh 38,000,000', 'KSh 16,500,000',
       'KSh 32,250,000', 'KSh 39,000,000', 'KSh 9,500,000',
       'KSh 28,000,000', 'KSh 8,950,000', 'KSh 50,600,000',
       'KSh 70,250,000', 'KSh 19,000,000', 'KSh 155,000,000',
       'KSh 180,000,000', 'KSh 38,500,000', 'KSh 5,300,000',
       'KSh 8,500,000', 'KSh 29,999,000', 'KSh 25,000,000',
       'KS

This column selling price is also a bit mixed. We have some of the readings as "Price not communicated" thus we need to come up with a strategy to handle this. Also later on, we shall remove the prefix "Ksh" which states the currency type to be left with only integers. Buckle up, this shall be a train ride.

# Data Preprocessing
   - Missing value treatment (if needed)
   - Feature engineering
   - Outlier detection and treatment (if needed)
   - Preparing data for modeling
   - Any other preprocessing steps (if needed)

Creating a copy of the dataset and working on the copy is a trend that not most people use. I find this trend very useful especially if we are working on a very crucial dataset. We might make a mistake, and this might cost us alot if we are working on the original piece. 
Work on the duplicate and once everything looks ood, you can choose to push the changes to the original dataset. This is just my view though.

In [7]:
# create copy of dataset
df1= df.copy()

In [8]:
import re
# Define a function to extract property type, number of bedrooms, and amenities from the title
def extract_info(title):
    # Extract the number of bedrooms
    bedrooms = re.search('(\d+) Bed', title)
    if bedrooms:
        bedrooms = bedrooms.group(1)
    else:
        bedrooms = 'Unknown'
    
    # Extract property type
    property_type = re.search('([a-zA-Z]+) with', title)
    if property_type:
        property_type = property_type.group(1)
    else:
        property_type = 'Unknown'
    
    # Extract amenities
    amenities = []
    for amenity in ['En Suite', 'Swimming Pool', 'Gym', 'Garden', 'Backup Generator', 'DSQ']:
        if amenity.lower() in title.lower():
            amenities.append(amenity)
    amenities_str = ', '.join(amenities) if amenities else 'Unknown'
    
    return bedrooms, property_type, amenities_str

# Apply the function to the 'title' column
df1[['bedrooms', 'property_type', 'amenities']] = df1['title'].apply(lambda x: pd.Series(extract_info(x)))


In [9]:
# Displaying the first few rows of the DataFrame
df1 = df1.drop(['title', 'size'], axis = 1)
df1

,location,selling price,bedrooms,property_type,amenities
0,Kiambu Road,"KSh 130,000,000",6,Villa,En Suite
1,"Off Kiambu Road, Kiambu Road","KSh 26,950,000",4,Townhouse,En Suite
2,"masai lodge road, Ongata Rongai","KSh 16,000,000",4,Villa,En Suite
3,Ngong Road,"KSh 45,000,000",5,House,En Suite
4,Lavington,"KSh 72,000,000",5,Townhouse,En Suite
...,...,...,...,...,...
1871,"Nandi road, Karen","KSh 199,000,000",5,House,En Suite
1872,Kitengela,"KSh 16,000,000",4,House,En Suite
1873,Syokimau,"KSh 13,800,000",4,House,En Suite
1874,Lavington,"KSh 79,000,000",4,Townhouse,Garden


In [10]:
# Remove 'KSh' prefix from 'selling_price'
df1['selling price'] = df1['selling price'].str.replace('KSh ', '')
df1

,location,selling price,bedrooms,property_type,amenities
0,Kiambu Road,"130,000,000",6,Villa,En Suite
1,"Off Kiambu Road, Kiambu Road","26,950,000",4,Townhouse,En Suite
2,"masai lodge road, Ongata Rongai","16,000,000",4,Villa,En Suite
3,Ngong Road,"45,000,000",5,House,En Suite
4,Lavington,"72,000,000",5,Townhouse,En Suite
...,...,...,...,...,...
1871,"Nandi road, Karen","199,000,000",5,House,En Suite
1872,Kitengela,"16,000,000",4,House,En Suite
1873,Syokimau,"13,800,000",4,House,En Suite
1874,Lavington,"79,000,000",4,Townhouse,Garden


We shall do stanerdization of capitalization to our feilds location, property_type and amenities.

Consistency: It helps maintain consistency in the dataset or document, making it easier to read and analyze.

Analysis: Standardization can make it easier to perform text analysis, such as searching for specific terms or conducting natural language processing tasks.

Comparison: It enables easier comparison between texts, as differences in capitalization won't affect the comparison.

Readability: Consistent capitalization can improve the overall readability of the text.

In [11]:
# Convert the 'location' column to lowercase
df1['location'] = df1['location'].str.lower()
df1['property_type'] = df1['property_type'].str.lower()
df1['amenities'] = df1['amenities'].str.lower()
df1

,location,selling price,bedrooms,property_type,amenities
0,kiambu road,"130,000,000",6,villa,en suite
1,"off kiambu road, kiambu road","26,950,000",4,townhouse,en suite
2,"masai lodge road, ongata rongai","16,000,000",4,villa,en suite
3,ngong road,"45,000,000",5,house,en suite
4,lavington,"72,000,000",5,townhouse,en suite
...,...,...,...,...,...
1871,"nandi road, karen","199,000,000",5,house,en suite
1872,kitengela,"16,000,000",4,house,en suite
1873,syokimau,"13,800,000",4,house,en suite
1874,lavington,"79,000,000",4,townhouse,garden


Remember when we were checking through our data for anomalies, we did notice that the selling price column had records where the prices were not mentioned and noted as "Price not communicated". There was
several ways to deal with this where one bwas to delete the records and work with the rest. Since i did not want to deleta any records from my data set, I had to amputate these records with the average values af houses that are located in the same location, and have a chance of having almost the same no. of rooms. This was quite hectic process since I had ri group the data into location and no.of  bedrooms then find the average, then create a new df where i could fix the mising records.

In [12]:
# Create a copy of the original DataFrame to work with
df2 = df1.copy()

# Remove commas from 'selling price' and convert to numeric, ignoring errors to handle 'Price not communicated'
df2['selling price'] = pd.to_numeric(df2['selling price'].str.replace(',', ''), errors='coerce')

# Group by 'location' and 'bedrooms', calculate the mean selling price
location_bedrooms_means = df2.groupby(['location', 'bedrooms'])['selling price'].mean()

# Fill missing 'selling_price' values based on the calculated means
for index, row in df2.iterrows():
    if pd.isnull(row['selling price']):  # Check if the selling price is missing
        location = row['location']
        bedrooms = row['bedrooms']
        # Use the mean for the location and closest number of bedrooms
        try:
            df2.at[index, 'selling price'] = location_bedrooms_means.loc[location, bedrooms]
        except KeyError:
            # If the exact number of bedrooms is not found, try the closest one
            bedrooms = min(location_bedrooms_means.loc[location].index, key=lambda x: abs(x - bedrooms))
            df2.at[index, 'selling price'] = location_bedrooms_means.loc[location, bedrooms]

# Convert 'selling_price' back to string format
df2['selling price'] = df2['selling price'].apply(lambda x: f"KSh {x:,.2f}")


In [13]:
# Display the df
df2

,location,selling price,bedrooms,property_type,amenities
0,kiambu road,"KSh 130,000,000.00",6,villa,en suite
1,"off kiambu road, kiambu road","KSh 26,950,000.00",4,townhouse,en suite
2,"masai lodge road, ongata rongai","KSh 16,000,000.00",4,villa,en suite
3,ngong road,"KSh 45,000,000.00",5,house,en suite
4,lavington,"KSh 72,000,000.00",5,townhouse,en suite
...,...,...,...,...,...
1871,"nandi road, karen","KSh 199,000,000.00",5,house,en suite
1872,kitengela,"KSh 16,000,000.00",4,house,en suite
1873,syokimau,"KSh 13,800,000.00",4,house,en suite
1874,lavington,"KSh 79,000,000.00",4,townhouse,garden


In [26]:
df2[df2['selling price'] == 'Price not communicated']

,location,selling price,bedrooms,property_type,amenities


In [27]:
df2['selling price'].unique()

array(['130000000', '26950000', '16000000', '45000000', '72000000',
       '46000000', '85000000', '50000000', '75000000', '500000000',
       '140000000', '15000000', '60000000', '55000000',
       'Pricenotcommunicated', '427000000', '65000000', '48000000',
       '88000000', '27000000', '150000000', '120000000', '110000000',
       '90000000', '35000000', '59000000', '95000000', '24000000',
       '20000000', '23000000', '115000000', '38000000', '16500000',
       '32250000', '39000000', '9500000', '28000000', '8950000',
       '50600000', '70250000', '19000000', '155000000', '180000000',
       '38500000', '5300000', '8500000', '29999000', '25000000',
       '72499999', '7200000', '6500000', '26500000', '17000000',
       '64990000', '70000000', '30500000', '100000000', '29500000',
       '15500000', '14500000', '7500000', '31500000', '16200000',
       '6800000', '11500000', '295000000', '10300000', '14000000',
       '22500000', '121000000', '8850000', '170000000', '99000000',
  

In [28]:
df2 = df2[df2['bedrooms'] != 'Unknown']
df2 = df2[df2['selling price'] != 'Pricenotcommunicated']
df2 = df2[df2['selling price'] != '']

In [29]:
df2['selling price'] = df1['selling price'].str.replace('KSh ', '')
df2

,location,selling price,bedrooms,property_type,amenities
0,kiambu road,"130,000,000",6,villa,en suite
1,"off kiambu road, kiambu road","26,950,000",4,townhouse,en suite
2,"masai lodge road, ongata rongai","16,000,000",4,villa,en suite
3,ngong road,"45,000,000",5,house,en suite
4,lavington,"72,000,000",5,townhouse,en suite
...,...,...,...,...,...
1871,"nandi road, karen","199,000,000",5,house,en suite
1872,kitengela,"16,000,000",4,house,en suite
1873,syokimau,"13,800,000",4,house,en suite
1874,lavington,"79,000,000",4,townhouse,garden


In [30]:
df2['bedrooms'].unique()

array(['6', '4', '5', '3', '8', '2', '7', '10', '1', '9'], dtype=object)

In [31]:
df2['selling price'].unique()

array(['130,000,000', '26,950,000', '16,000,000', '45,000,000',
       '72,000,000', '46,000,000', '85,000,000', '50,000,000',
       '75,000,000', '500,000,000', '140,000,000', '15,000,000',
       '60,000,000', '55,000,000', '427,000,000', '65,000,000',
       '48,000,000', '88,000,000', '27,000,000', '150,000,000',
       '120,000,000', '110,000,000', '90,000,000', '35,000,000',
       '59,000,000', '95,000,000', '24,000,000', '20,000,000',
       '23,000,000', '115,000,000', '38,000,000', '16,500,000',
       '32,250,000', '39,000,000', '9,500,000', '28,000,000', '8,950,000',
       '50,600,000', '70,250,000', '19,000,000', '155,000,000',
       '180,000,000', '38,500,000', '5,300,000', '8,500,000',
       '29,999,000', '25,000,000', '72,499,999', '7,200,000', '6,500,000',
       '26,500,000', '17,000,000', '64,990,000', '70,000,000',
       '30,500,000', '100,000,000', '29,500,000', '15,500,000',
       '14,500,000', '7,500,000', '31,500,000', '16,200,000', '6,800,000',
       '11

In [35]:
# Remove white spaces and commas from values in the 'selling price' column
df2['selling price'] = df2['selling price'].astype(str).str.replace(' ', '').str.replace(',', '')

# Convert the 'selling price' column to integer
df2['selling price'] = df2['selling price'].astype(int)

# Convert the 'bedrooms' column to integer
df2['bedrooms'] = df2['bedrooms'].astype(int)


# Model Bulding - LinearRegression

In [53]:
# # LABEL ENCODING
# df2 = pd.get_dummies(df2, columns=['location', 'property_type', 'amenities'], drop_first=True, dtype='int8')
# df2
df2

,selling price,bedrooms,"location_01, ongata rongai","location_01, runda, runda, westlands","location_01, runda, westlands","location_012, kimlat, kitengela","location_0208, ngong road, ngong","location_04, gateway mall, syokimau","location_046, kindaruma road, kilimani","location_0706771810, bamburi mwisho, bamburi",...,"location_yukos, kitengela",property_type_townhouse,property_type_unknown,property_type_villa,amenities_en suite,"amenities_en suite, garden",amenities_garden,amenities_gym,amenities_swimming pool,amenities_unknown
0,130000000,6,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,26950000,4,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,16000000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,45000000,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,72000000,5,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,199000000,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1872,16000000,4,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1873,13800000,4,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1874,79000000,4,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [54]:
y = df2['selling price']
X = df2.drop('selling price', axis =1)

In [55]:
#feature engineering
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)


In [56]:
#feature scalling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)


In [58]:
#gradient boosting
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)


GradientBoostingRegressor()

In [59]:
# rifge regression
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

Ridge()

In [60]:
#model selection
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)


RandomForestRegressor()

In [61]:
print("Number of rows in train data =", X_train.shape[0])
print("Number of rows in test data =", X_test.shape[0])

Number of rows in train data = 1477
Number of rows in test data = 370


In [62]:
lm = LinearRegression()

In [63]:
#hyperparameter Tuning
from sklearn.model_selection import GridSearchCV
params = {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, 30]}
grid_search = GridSearchCV(rf, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 20, 30],
                         'n_estimators': [100, 200, 300]})

In [65]:
lm.fit(X_train, y_train)

LinearRegression()

In [66]:
#crosss validation (k-fold)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')


In [ ]:
# Extract the intercept, or y-cut, of our linear model
a = float(lm.intercept_)
# Extract the coefficient, or gradient, of our linear model
b = lm.coef_

In [ ]:
print("Slope:\t\t", b)
print("Intercept:\t", float(a))

# Model performance check

In [67]:
# Generate predictions on the training set
y_train_pred = lm.predict(X_train)

# Calculate the Mean Squared Error (MSE)
train_mse = metrics.mean_squared_error(y_train, y_train_pred)

# Calculate the R-squared
train_r2 = metrics.r2_score(y_train, y_train_pred)

# Print the training MSE and R-squared score
print("Training MSE:", train_mse)
print("Training R-squared:", train_r2)

Training MSE: 1099726801155672.2
Training R-squared: 0.7820668706257982


In [68]:
# Your solution here...
# Generate predictions on the testing set
y_test_pred = lm.predict(X_test)

# Calculate the Mean Squared Error (MSE)
test_mse = metrics.mean_squared_error(y_test, y_test_pred)

# Calculate the R-squared
test_r2 = metrics.r2_score(y_test, y_test_pred)

# Print the testing MSE and R-squared score
print("Testing MSE:", test_mse)
print("Testing R-squared:", test_r2)

Testing MSE: 1.4153579376362048e+40
Testing R-squared: -2.8837743267611474e+24


In [69]:
# Make predictions on the test data
y_pred = lm.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 1.4153579376362048e+40
R^2 Score: -2.8837743267611474e+24
